In [1]:
import matplotlib.pyplot as plt, numpy as np, numpy.random as npr, csv, matplotlib.pyplot as plt, pandas as pd
import torch.nn as nn, timeit, torch, pystan as ps, scipy, seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import LocallyLinearEmbedding
from tqdm import trange
%matplotlib inline
from time import time
from pylab import plot, show, legend

In [2]:
sm = ps.StanModel(file="/home/postdoc/dsen/Desktop/codes/BNN_dimension_reduction/stan_files/nn_refpoints_constrainedtheta.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cf61588642e0539b092dedc55e934141 NOW.


In [3]:
inp_data = pd.read_csv("data_banknote_authentication.txt", header=0).values
X = inp_data[:,:4].transpose()
Nobs = np.shape(X)[1]
print("Nobs =", Nobs, ", dim =", np.shape(X)[0])

Nobs = 1371 , dim = 4


In [4]:
0.06*Nobs

82.25999999999999

In [5]:
n_ref = 80
lat_dim = 2
obs_dim = np.shape(X)[0] 
ref_points = npr.choice(Nobs,n_ref,replace=False)
embedding = LocallyLinearEmbedding(n_components=lat_dim, n_neighbors=10)
X_embedded = embedding.fit_transform(X.transpose()).transpose()

In [6]:
train = np.setdiff1d(np.arange(Nobs), ref_points)
X_ref = X[:,ref_points]
Z_ref = X_embedded[:,ref_points]
X_train = X[:,train]

In [7]:
n_in = np.shape(X_embedded)[0]
n_h = 40
n_out = np.shape(X)[0]

model = nn.Sequential(nn.Linear(n_in, n_h), nn.Tanh(), nn.Linear(n_h, n_out))

In [8]:
def init_normal(m):
    if type(m) == nn.Linear: nn.init.normal_(m.weight)
model.apply(init_normal)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)

In [9]:
n_iter = 500_000
Lss = np.zeros(n_iter)
start = time()
for epoch in range(n_iter):
    X_pred = model(torch.from_numpy(Z_ref.transpose().astype(np.float32)))
    loss = criterion(X_pred, torch.from_numpy(X_ref.transpose().astype(np.float32)))
    Lss[epoch] = loss.data
    if (epoch+1)%(n_iter/10) == 0 :
        print("Epoch: %d; loss: %.3f; time: %.2f mins" % (epoch+1, loss.item(), (time()-start)/60))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 50000; loss: 14.257; time: 0.27 mins
Epoch: 100000; loss: 11.214; time: 0.53 mins
Epoch: 150000; loss: 8.221; time: 0.79 mins
Epoch: 200000; loss: 7.964; time: 1.06 mins
Epoch: 250000; loss: 7.890; time: 1.33 mins
Epoch: 300000; loss: 7.839; time: 1.59 mins
Epoch: 350000; loss: 7.795; time: 1.86 mins
Epoch: 400000; loss: 7.759; time: 2.14 mins
Epoch: 450000; loss: 7.732; time: 2.41 mins
Epoch: 500000; loss: 7.711; time: 2.68 mins


In [10]:
for (i,p) in enumerate(model.parameters()):
    if i == 0 : W_1_optim = p.data;
    print(np.shape(p))

torch.Size([40, 2])
torch.Size([40])
torch.Size([4, 40])
torch.Size([4])


In [ ]:
c = [np.linalg.norm(W_1_optim[i,:]) for i in range(n_in)]
for i in range(n_ref) : Z_ref[:,i] *= c

### Run Stan:

In [ ]:
data = dict(Nobs=Nobs, in_dim=lat_dim, out_dim=n_out, hidden_dim=n_h, n_ref=n_ref, X=X_train, X_ref=X_ref, Z_ref=Z_ref)
start = time()
fit = sm.sampling(data=data, thin=1, n_jobs=10, chains=10, init="random", iter=1_000)
print(round((time()-start)/60,2), "minutes to run")
print(fit);

In [25]:
trace = fit.extract()
np.save('/xtmp/BNN_dim_red_data/stan_fit_banknote.npy', trace) 
n_samples, n_latent = np.shape(trace['Z'])[0], np.shape(trace['Z'])[1]

#### Traceplots of latent variables:

In [ ]:
plt.figure(figsize=(10,8))
idx = npr.choice(n_latent,25).reshape(5,5)

for i in range(5):
    for j in range(5):
        k = 5*i+j
        plt.subplot(5,5,k+1)
        plt.plot(trace['Z'][:,idx[i,j],0])
        if i < 4 : plt.xticks(alpha=0)
        if i == 4 : plt.xlabel("MCMC sample", fontsize=12)
        plt.grid(True)
plt.suptitle("Traceplots of some latent variables", fontsize=15, x=0.5,y=0.95)
plt.savefig('/home/postdoc/dsen/Desktop/codes/BNN_dimension_reduction/plots/banknote_traceplot_constrainedW1_refpoints.pdf', 
                dpi=2000, bbox_inches='tight');

### Loss:

In [ ]:
def L2_loss(X, Z, weights_1, bias_1, weights_2, bias_2) :
    X_pred = np.matmul(np.tanh(np.matmul(Z,weights_1)+bias_1), weights_2) + bias_2
    return np.mean(np.sum((X-X_pred)**2,axis=1))

In [ ]:
loss = np.zeros(n_samples)
for i in trange(n_samples) :
    loss[i] = L2_loss(X_train.transpose(), trace['Z'][i], trace['weights_1'][i], 
                      trace['bias_1'][i], trace['weights_2'][i], trace['bias_2'][i])
plt.figure(figsize=(7,4))
plt.plot(loss)
plt.title("Loss", fontsize=15)
plt.xlabel("MCMC sample", fontsize=13)
plt.grid(True)

### Pairwise distances:

In [ ]:
n_samples, n_latent = np.shape(trace['Z'])[0], np.shape(trace['Z'])[1]
k_max = 50

pw_dist_matrix = np.zeros((k_max,k_max,n_samples))
for k in trange(n_samples) :
    for i in range(k_max) :
        for j in range(k_max) :
            pw_dist_matrix[i,j,k] = np.linalg.norm(trace['Z'][k,i]-trace['Z'][k,j])

In [ ]:
k_max = 50
idx = npr.choice(k_max,10,replace=False)
plt.figure(figsize=(20,15))
for i in range(6):
    for j in range(6):
        if i > j :
            k = 10*i+j+1
            plt.subplot(10,10,k)
            plt.plot(pw_dist_matrix[idx[i],idx[j]])
            if i < 5 : plt.xticks(alpha=0)
            plt.grid(True)
            if i == 5 : plt.xlabel("MCMC sample", fontsize=12)
            if j > 0 : plt.yticks(alpha=0)
plt.suptitle("Traceplots of some pairwise distances for banknote data", fontsize=15, x=0.31,y=0.83)
# plt.savefig('/home/postdoc/dsen/Desktop/codes/BNN_dimension_reduction/plots/banknote_pwdist_constrainedW1_refpoints.pdf', 
#                 dpi=2000, bbox_inches='tight');

### Summaries of pairwise distances:

In [ ]:
latent_samples = np.zeros((Nobs,n_samples,lat_dim))
k = 0
ref_points_sorted = np.sort(ref_points)
Z_ref_sorted = Z_ref[:,np.argsort(ref_points)]
for i in trange(Nobs) :
    if k < len(ref_points_sorted) :
        if i == ref_points_sorted[k] :
            latent_samples[i] = Z_ref[:,k]
            k += 1 
        else :
            latent_samples[i] = trace['Z'][:,i-k]

In [ ]:
pw_dist_summary = np.zeros((4,Nobs,Nobs))
for i in trange(Nobs) :
    for j in range(Nobs) :
        pwdist = scipy.linalg.norm(latent_samples[i]-latent_samples[j], axis=1)
        pw_dist_summary[0,i,j] = np.mean(pwdist)
        pw_dist_summary[1,i,j] = np.median(pwdist)
        pw_dist_summary[2,i,j] = np.percentile(pwdist,q=2.5)
        pw_dist_summary[3,i,j] = np.percentile(pwdist,q=97.5)

In [ ]:
titles = ["Mean", "Median", "Credible interval lower limit", "Credible interval upper limit"]
fig = plt.figure(figsize=(15,12))
for i in range(4) :
    plt.subplot(2,2,i+1)
    ax = sns.heatmap(pw_dist_summary[i])
    plt.title(titles[i])
plt.subplots_adjust(wspace=1e-2)
# plt.savefig('/xtmp/BNN_dim_red_data/banknote_pwdist_heatmap.pdf', dpi=1000, box_inches='tight')
# plt.savefig('/xtmp/BNN_dim_red_data/banknote_pwdist_heatmap.jpeg', dpi=700, box_inches='tight')
plt.show()

In [24]:
np.save('/xtmp/BNN_dim_red_data/banknote_pw_dist_summary.npy', pw_dist_summary)
np.save('/xtmp/BNN_dim_red_data/banknote_latent_samples.npy', latent_samples)